In [12]:
from agno.models.openai import OpenAILike
from agno.agent import Agent
import uuid
import requests
from dotenv import load_dotenv
import os
from agno.os import AgentOS

# Load environment variables from .env file
load_dotenv()

True

In [16]:
from pydantic_settings import BaseSettings
from pydantic import Field


class Settings(BaseSettings):
    """Application settings loaded from environment variables"""

    client_app_secret: str = Field(
        ..., description="Client application secret for authentication"
    )
    experience_id: str = Field(..., description="Intuit experience ID")
    client_app_id: str = Field(..., description="Client application ID")
    profile_id: str = Field(..., description="Profile ID for authentication")

    # Pydantic v2 uses model_config instead of inner Config class
    model_config = {
        "env_file": ".env",
        "env_file_encoding": "utf-8",
        "case_sensitive": False,
        "extra": "ignore",  # Ignore extra env vars like SERPER_API_KEY
    }


# Initialize settings
settings = Settings()

In [14]:
def _get_auth_token():
    """Get cached authentication token or fetch a new one if not available"""

    # Fetch new token from IAM
    url = "https://identityinternal.api.intuit.com/v1/graphql"

    IAM_MUTATION = """mutation identitySignInInternalApplicationWithPrivateAuth($input: Identity_SignInApplicationWithPrivateAuthInput!) {
        identitySignInInternalApplicationWithPrivateAuth(input: $input) {
            authorizationHeader
        }
    }"""

    headers = {
        "intuit_tid": str(uuid.uuid4()),
        "Authorization": f"Intuit_IAM_Authentication intuit_appid={settings.client_app_id}, intuit_app_secret={settings.client_app_secret}",
        "Content-Type": "application/json",
    }

    data = {
        "query": IAM_MUTATION,
        "variables": {"input": {"profileId": settings.profile_id}},
    }

    try:
        response = requests.post(url, json=data, headers=headers)
        response.raise_for_status()
        result = response.json()

        base_token = result["data"]["identitySignInInternalApplicationWithPrivateAuth"][
            "authorizationHeader"
        ]
        _cached_token = f"{base_token},intuit_appid={settings.client_app_id},intuit_app_secret={settings.client_app_secret}"
        print("✓ New auth token obtained and cached")
        return _cached_token
    except Exception as e:
        print(f"✗ Failed to get auth token: {e}")
        raise

In [25]:
llm = OpenAILike(
    base_url="https://llmexecution.api.intuit.com/v3/lt/amazon.nova-pro-v1-0",
    extra_headers={
        "intuit_experience_id": settings.experience_id,
        "intuit_originating_assetalias": "Intuit.coe.pecomplianceremediation",
        "Authorization": _get_auth_token(),
    },
)

✓ New auth token obtained and cached


In [18]:
gen_agent = Agent(
    role="Generalist",
    instructions="You are a generalist agent that can help with a wide range of tasks.",
    model=llm,
)

In [19]:
response = gen_agent.run("who are you?")

In [20]:
response.to_dict()

{'run_id': '260b57c0-d8d3-43f6-a9ee-d0e7512294b7',
 'agent_id': 'b19f47fd-5a99-450e-812a-fd6822152cdd',
 'session_id': '0450033f-b738-465f-b20e-9279582e8ab9',
 'content': "I'm Claude, an AI assistant created by Anthropic. I'm designed to be helpful, harmless, and honest in my interactions. As a generalist AI, I can assist with a wide variety of tasks including:\n\n- Answering questions and explaining concepts\n- Writing and editing text\n- Analysis and research\n- Problem-solving and brainstorming\n- Creative tasks like storytelling or ideation\n- Math and coding help\n- And many other areas\n\nI aim to be thoughtful and nuanced in my responses while being direct and clear about what I can and cannot do. Is there something specific I can help you with today?",
 'content_type': 'str',
 'model_provider_data': {'id': 'chatcmpl-c1c68601-f18a-48b3-b075-b8380f2469ba'},
 'model': 'not-provided',
 'model_provider': 'OpenAI',
 'session_state': {'current_session_id': '0450033f-b738-465f-b20e-927

In [17]:
agent_os = AgentOS(
    id="my-first-os",
    description="My first AgentOS",
    agents=[gen_agent],
)

app = agent_os.get_app()
agent_os.serve(app="my_os:app", reload=True)

╔═══════════════ AgentOS ════════════════╗
║                                        ║
║                                        ║
║          https://os.agno.com/          ║
║                                        ║
║                                        ║
║  OS running on: http://localhost:7777  ║
║                                        ║
║                                        ║
╚════════════════════════════════════════╝

INFO:     Will watch for changes in these directories: ['/Users/ssahu11/Personal/BITS/Sem4/Dessertation/FinAgent']
INFO:     Uvicorn running on http://localhost:7777 (Press CTRL+C to quit)
INFO:     Started reloader process [30060] using WatchFiles
ERROR:    Error loading ASGI app. Could not import module "my_os".
INFO:     Stopping reloader process [30060]


In [5]:
from agno.vectordb.chroma import ChromaDb
from agno.knowledge import Knowledge, embedder
from agno.knowledge.embedder.sentence_transformer import SentenceTransformerEmbedder
from agno.knowledge.chunking.fixed import FixedSizeChunking
from agno.knowledge.reader.pdf_reader import PDFReader

kn = Knowledge(
    name="income tax documents",
    description="This is a knowledge base for income tax documents",
    vector_db=ChromaDb(
        collection="tax_docs",
        path="tmp/chroma",
        persistent_client=True,
        embedder=SentenceTransformerEmbedder(id="all-MiniLM-L6-v2"),
    ),
)

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2299.10it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [7]:
reader = PDFReader(
    chunk_size=1000, chunking_strategy=FixedSizeChunking(chunk_size=1000, overlap=200)
)

In [8]:
kn2 = Knowledge(
    name="Mid Sem Report",
    description="This is a knowledge base for Mid Sem Report",
    vector_db=ChromaDb(
        collection="midsem_report",
        path="tmp/chroma2",
        persistent_client=True,
        embedder=SentenceTransformerEmbedder(id="all-MiniLM-L6-v2"),
    ),
)

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2059.72it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [10]:
reader = PDFReader(
    chunk_size=1000, chunking_strategy=FixedSizeChunking(chunk_size=1000, overlap=200)
)


kn.add_content(
    path="/Users/ssahu11/Personal/BITS/Sem4/Dessertation/MidSem_Report_2023ac05804.pdf",
    reader=reader,
    metadata={"type": "midsem_report"},
)

INFO Adding content from path, a30137a4-6d12-5c4d-8b02-15bfbf5b0ec4, None,      
     /Users/ssahu11/Personal/BITS/Sem4/Dessertation/MidSem_Report_2023ac05804.pd
     f, None                                                                    
INFO Upserting 41 documents                                                     
WARNING  Contents DB not found for knowledge base: income tax documents         


In [26]:
from agno.agent import Agent

kn_agent = Agent(
    role="Knowledge Base Agent",
    instructions="You are a knowledge base agent that can help with a wide range of tasks.",
    model=llm,
    knowledge=kn,
    search_knowledge=True,
    knowledge_filters={"type": "midsem_report"},
)

In [27]:
kn_agent.run("what is this project about?").content

INFO Found 10 documents                                                         


'\nThe project aims to develop a "Financial Guardian" system that leverages Large Language Models (LLMs) and Multi-Agent Orchestration frameworks to provide personalized financial advice. The system features a dual-team architecture: a "Personal Finance Team" for budget and tax compliance, and an "Investment Helper Team" for wealth accumulation. By integrating LLMs with a robust Vector Database knowledge base of Indian financial rules, the project seeks to democratize access to sophisticated financial guardianship, effectively bridging the gap between financial products and consumer capability.\n\nKey components of the project include:\n- A structured onboarding process to collect user data.\n- A conversational chat interface for user interaction.\n- A modern, Python-centric tech stack optimized for agentic workflows and advanced reasoning.\n- A dual-LLM strategy using OpenAI GPT-5.1 and Anthropic Claude 4.5 Sonnet for tool calling and complex reasoning.\n\nThe project is currently in 